In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import copy
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import networkx as nx
from torch.nn.utils import parameters_to_vector, vector_to_parameters

In [2]:
# Graph implementation
def generate_graph(cluster_sizes=[100,100], pin=0.5, pout=0.01, seed=0):
    """Generate a random connected graph"""
    probs = np.array([[pin, pout],[pout, pin]])
    while True:
        g = nx.stochastic_block_model(cluster_sizes, probs)
        if nx.algorithms.components.is_connected(g):
            return g


cluster_sizes = [10, 10]
pin = 0.5
#pout = 0.01
pout = 0.4
seed = 0
alpha = 1e-3
lamda = 1e-3
eta = 1e-3
no_users = sum(cluster_sizes)
batch_size = 20
epochs = 1
it = 2000
G = generate_graph(cluster_sizes, pin, pout, seed)

#nx.draw(G, with_labels=True, node_size=100, alpha=1, linewidths=10)
#plt.show()

In [3]:
# Metropolis weights 
number_nodes = G.number_of_nodes()
weights = np.zeros([number_nodes, number_nodes])
for edge in G.edges():
  i, j = edge[0], edge[1]
  weights[i - 1][j - 1] = 1 / (1 + np.max([G.degree(i), G.degree(j)]))
  weights[j - 1][i - 1] = weights[i - 1][j - 1]

print(weights)

weights = weights + np.diag(1 - np.sum(weights, axis=0))

metropolis_weights = weights
print(metropolis_weights)


[[0.         0.         0.         0.125      0.07692308 0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.14285714 0.         0.125      0.
  0.         0.        ]
 [0.         0.         0.         0.125      0.07692308 0.
  0.         0.125      0.125      0.         0.         0.
  0.07142857 0.         0.         0.         0.         0.1
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.07142857 0.1        0.         0.14285714 0.         0.
  0.         0.16666667]
 [0.125      0.125      0.         0.         0.         0.125
  0.1        0.         0.125      0.         0.         0.
  0.07142857 0.         0.125      0.         0.         0.
  0.         0.        ]
 [0.07692308 0.07692308 0.         0.         0.         0.07692308
  0.07692308 0.07692308 0.07692308 0.07692308 0.07692308 0.
  0.07142857 0.         0.         0.         0.

In [4]:
def load_dataset():
    transforms_mnist = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,),(0.3081,))])
    mnist_data_train = datasets.MNIST('./data/mnist', train=True, download=True, transform=transforms_mnist)
    mnist_data_test = datasets.MNIST('./data/mnist', train=False, download=True, transform=transforms_mnist)

    return mnist_data_train, mnist_data_test

In [5]:
def degrees(A):
    """Return the degrees of each node of a graph from its adjacency matrix"""
    return np.sum(A, axis=0).reshape(A.shape[0], 1)

def node_degree(n, G):
    cnt = 0
    for i in G.neighbors(n):
        cnt += 1
    return cnt

def get_neighbors(n, G):
    neighbors_list = []
    for i in G.neighbors(n):
        neighbors_list.append(int(i))
    return neighbors_list

In [6]:
datapoints = {}
count = 0
W1 = np.array([2.0, 2.0, 3.0, 3.0])
W2 = np.array([-2.0, 2.0, 3.0, -3.0])
W3 = 2 * W1
W4 = 2  * W2
W = [W1, W2]
m = 200
n = 4
noise_sd = 0.001
for i, cluster_size in enumerate(cluster_sizes):
    for j in range(cluster_size):
        features = np.random.normal(loc=0.0, scale=1.0, size=(m, n))
        label = np.dot(features, W[i ]) + np.random.normal(0,noise_sd)
        datapoints[count] = {
                'features': features,
                'degree': node_degree(count, G),
                'label': label,
                'neighbors': get_neighbors(count, G),
                'exact_weights': torch.from_numpy(W[i])
            }
        count += 1

In [7]:
class MyDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = torch.FloatTensor(data)
        self.targets = torch.FloatTensor(targets).unsqueeze(-1)
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]

        return x, y
    
    def __len__(self):
        return len(self.data)


In [8]:
class MLP_Net(nn.Module):
    def __init__(self, user_id):
        super(MLP_Net, self).__init__()
        self.fc1 = nn.Linear(4, 1, bias=False)
        #self.fc2 = nn.Linear(2, 1, bias=False)
        #self.fc3 = nn.Linear(200, 10)
        self.user_id = user_id

    def forward(self, x):
        x = torch.flatten(x, 1)
        #x = F.relu(self.fc1(x))
        output = self.fc1(x)
        #output = self.fc3(x)
        return output

In [9]:
from typing import Iterable, Optional

def grads_to_vector(parameters: Iterable[torch.Tensor]) -> torch.Tensor:
    r"""Convert parameters to one vector

    Args:
        parameters (Iterable[Tensor]): an iterator of Tensors that are the
            parameters of a model.

    Returns:
        The parameters represented by a single vector
    """
    # Flag for the device where the parameter is located
    param_device = None

    vec = []
    for param in parameters:
        # Ensure the parameters are located in the same device
        param_device = param.grad

        vec.append(param_device.view(-1))
    return torch.cat(vec)

In [10]:
model = MLP_Net(user_id=0)

lr = 0.01

dataloader = DataLoader(MyDataset(datapoints[19]["features"], datapoints[19]["label"]), batch_size=50, shuffle=False)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
for i in range(100):
    for (x, y) in dataloader:
        criterion = nn.MSELoss()
        optimizer.zero_grad()
        yhat = model(x)
        print(y.size())
        print(yhat.size())
        loss = criterion(yhat, y)
        
        loss.backward()
        print(i, loss, grads_to_vector(model.parameters()), parameters_to_vector(model.parameters()))
        #optimizer.step()
        new_model = parameters_to_vector(model.parameters()) - lr * grads_to_vector(model.parameters())
        vector_to_parameters(parameters=model.parameters(), vec=new_model)
        #if i % 50 ==0:
            #lr *= 0.9
            

#parameters_to_vector(model.parameters())

torch.Size([50, 1])
torch.Size([50, 1])
0 tensor(35.6923, grad_fn=<MseLossBackward0>) tensor([  4.5316,  -4.0697, -10.5881,   6.9526]) tensor([-0.2369, -0.2935, -0.3029, -0.2542], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
0 tensor(32.2348, grad_fn=<MseLossBackward0>) tensor([  4.0152,  -3.7878, -10.0506,   6.3183]) tensor([-0.2822, -0.2528, -0.1970, -0.3237], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
0 tensor(23.0703, grad_fn=<MseLossBackward0>) tensor([ 2.8854, -4.2088, -7.0069,  3.9343]) tensor([-0.3223, -0.2149, -0.0965, -0.3869], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
0 tensor(30.5892, grad_fn=<MseLossBackward0>) tensor([ 5.0570, -7.5637, -7.5314,  5.2876]) tensor([-0.3512, -0.1728, -0.0264, -0.4263], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
1 tensor(29.1874, grad_fn=<MseLossBackward0>) tensor([ 4.0955, -3.7102, -9.5116,  6.3387]) tensor([-0.4018, -0.0972,  0.0489, -0.4791], grad_fn=<CatBack

18 tensor(0.9216, grad_fn=<MseLossBackward0>) tensor([ 0.6344, -0.7264, -1.4123,  1.2723]) tensor([-1.6962,  1.5274,  2.6229, -2.3909], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
18 tensor(0.6882, grad_fn=<MseLossBackward0>) tensor([ 0.4715, -0.7948, -0.9023,  0.9034]) tensor([-1.7025,  1.5347,  2.6370, -2.4036], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
18 tensor(0.9987, grad_fn=<MseLossBackward0>) tensor([ 0.9387, -1.5383, -0.9229,  1.1774]) tensor([-1.7073,  1.5427,  2.6461, -2.4127], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
19 tensor(0.8514, grad_fn=<MseLossBackward0>) tensor([ 0.6761, -0.7350, -1.3240,  1.2685]) tensor([-1.7166,  1.5580,  2.6553, -2.4244], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
19 tensor(0.7614, grad_fn=<MseLossBackward0>) tensor([ 0.5729, -0.6644, -1.2626,  1.1677]) tensor([-1.7234,  1.5654,  2.6685, -2.4371], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])


38 tensor(0.0253, grad_fn=<MseLossBackward0>) tensor([ 0.0843, -0.1293, -0.1323,  0.2451]) tensor([-1.9505,  1.9059,  2.9880, -2.8668], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
38 tensor(0.0215, grad_fn=<MseLossBackward0>) tensor([ 0.0669, -0.1395, -0.0577,  0.1992]) tensor([-1.9514,  1.9072,  2.9893, -2.8693], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
38 tensor(0.0340, grad_fn=<MseLossBackward0>) tensor([ 0.1632, -0.2971, -0.0429,  0.2526]) tensor([-1.9520,  1.9086,  2.9899, -2.8713], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
39 tensor(0.0242, grad_fn=<MseLossBackward0>) tensor([ 0.0981, -0.1361, -0.1205,  0.2426]) tensor([-1.9537,  1.9116,  2.9903, -2.8738], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
39 tensor(0.0214, grad_fn=<MseLossBackward0>) tensor([ 0.0763, -0.1190, -0.1162,  0.2266]) tensor([-1.9546,  1.9129,  2.9915, -2.8762], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])


57 tensor(0.0014, grad_fn=<MseLossBackward0>) tensor([ 0.0133, -0.0281, -0.0049,  0.0578]) tensor([-1.9899,  1.9775,  3.0083, -2.9657], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
57 tensor(0.0013, grad_fn=<MseLossBackward0>) tensor([ 0.0116, -0.0302,  0.0080,  0.0517]) tensor([-1.9900,  1.9778,  3.0084, -2.9662], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
57 tensor(0.0021, grad_fn=<MseLossBackward0>) tensor([ 0.0352, -0.0700,  0.0145,  0.0647]) tensor([-1.9902,  1.9781,  3.0083, -2.9667], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
58 tensor(0.0013, grad_fn=<MseLossBackward0>) tensor([ 0.0177, -0.0304, -0.0036,  0.0568]) tensor([-1.9905,  1.9788,  3.0081, -2.9674], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
58 tensor(0.0012, grad_fn=<MseLossBackward0>) tensor([ 0.0121, -0.0260, -0.0035,  0.0537]) tensor([-1.9907,  1.9791,  3.0082, -2.9680], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])


torch.Size([50, 1])
torch.Size([50, 1])
76 tensor(0.0002, grad_fn=<MseLossBackward0>) tensor([ 0.0083, -0.0181,  0.0077,  0.0178]) tensor([-1.9978,  1.9944,  3.0039, -2.9910], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
77 tensor(9.4312e-05, grad_fn=<MseLossBackward0>) tensor([ 0.0038, -0.0072,  0.0032,  0.0144]) tensor([-1.9978,  1.9946,  3.0038, -2.9911], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
77 tensor(8.9988e-05, grad_fn=<MseLossBackward0>) tensor([ 0.0023, -0.0062,  0.0036,  0.0138]) tensor([-1.9979,  1.9947,  3.0038, -2.9913], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
77 tensor(9.1646e-05, grad_fn=<MseLossBackward0>) tensor([ 0.0021, -0.0069,  0.0057,  0.0132]) tensor([-1.9979,  1.9947,  3.0038, -2.9914], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
77 tensor(0.0001, grad_fn=<MseLossBackward0>) tensor([ 0.0077, -0.0169,  0.0073,  0.0167]) tensor([-1.9979,  1.9948,  3.0037, -2.9916], grad_fn=<Cat

96 tensor(9.8336e-06, grad_fn=<MseLossBackward0>) tensor([ 0.0004, -0.0020,  0.0022,  0.0037]) tensor([-1.9995,  1.9987,  3.0012, -2.9976], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
96 tensor(1.3875e-05, grad_fn=<MseLossBackward0>) tensor([ 0.0018, -0.0048,  0.0022,  0.0050]) tensor([-1.9995,  1.9987,  3.0012, -2.9977], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
97 tensor(7.6932e-06, grad_fn=<MseLossBackward0>) tensor([ 0.0010, -0.0014,  0.0013,  0.0034]) tensor([-1.9995,  1.9988,  3.0012, -2.9977], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
97 tensor(8.7373e-06, grad_fn=<MseLossBackward0>) tensor([ 0.0006, -0.0015,  0.0020,  0.0034]) tensor([-1.9996,  1.9988,  3.0011, -2.9977], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
97 tensor(8.9158e-06, grad_fn=<MseLossBackward0>) tensor([ 0.0004, -0.0019,  0.0021,  0.0034]) tensor([-1.9996,  1.9988,  3.0011, -2.9978], grad_fn=<CatBackward0>)
torch.Size([50, 1])


In [11]:
parameters_to_vector(model.parameters())

tensor([-1.9996,  1.9990,  3.0009, -2.9981], grad_fn=<CatBackward0>)

In [12]:
class CNN_Net(nn.Module):
    def __init__(self):
        super(CNN_Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.pool = nn.MaxPool2d(2,2)
        self.dropout = nn.Dropout(p=0.2)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        output = self.fc2(x)
        return output

In [13]:
class ClientUpdate(object):
    def __init__(self, dataset, batchSize, alpha, lamda, epochs, projection_list, projected_weights):
        self.train_loader = DataLoader(MyDataset(dataset["features"], dataset["label"]), batch_size=batchSize, shuffle=True)
        #self.learning_rate = learning_rate
        self.epochs = epochs
        self.batchSize = batchSize

    def train(self, model):
        criterion = nn.MSELoss()
        optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.5)

        e_loss = []
        for epoch in range(1, self.epochs+1):
            train_loss = 0
            model.train()
            for i, (data, labels) in zip(range(1), self.train_loader):
                data, labels = data, labels
                optimizer.zero_grad() 
                output = model(data)  
                loss = criterion(output, labels)
                #loss += mu/2 * torch.norm(client_param.data - server_param.data)**2
                loss.backward()
                grads = grads_to_vector(model.parameters())
                #optimizer.step()
                train_loss += loss.item()*data.size(0)
                weights = parameters_to_vector(model.parameters())
                mat_vec_sum = torch.zeros_like(weights)
                for j in G.neighbors(model.user_id):
                    mat_vec_sum = torch.add(mat_vec_sum, torch.matmul(torch.transpose(projection_list[model.user_id][j], 0, 1), 
                                                         projected_weights[j][model.user_id] - projected_weights[model.user_id][j]))
                
                model_update = parameters_to_vector(model.parameters()) - alpha * (grads + lamda * mat_vec_sum)
                
            vector_to_parameters(parameters=model.parameters(), vec=model_update)
                

            train_loss = train_loss/self.batchSize#len(self.train_loader.dataset) 
            e_loss.append(train_loss)

        total_loss = e_loss#sum(e_loss)/len(e_loss)

        return model.state_dict(), total_loss

In [14]:
# Preparing projection matrices
models = [MLP_Net(user_id=i) for i in range(no_users)]
#temp = MLP_Net()
projection_list = []
projected_weights = []

def update_ProjWeight(projection_list, projected_weights, first_run=True):
    #projected_weights = []
    for i in range(no_users):
        neighbors_mat = []
        neighbors_weights = []
        for j in range(no_users):
            if j in G.neighbors(i):
                with torch.no_grad():
                    if first_run == True:
                        row, column = parameters_to_vector(models[j].parameters()).size()[0], parameters_to_vector(models[i].parameters()).size()[0]
                        mat = torch.zeros((row, column))
                        mat.fill_diagonal_(1.0 + 1.0 * float(np.random.randn(1)))
                        neighbors_mat.append(mat)
                        neighbors_weights.append(torch.matmul(mat, parameters_to_vector(models[j].parameters())))
                    else:
                        neighbors_weights.append(torch.matmul(projection_list[j][i], parameters_to_vector(models[j].parameters())))
            else:
                neighbors_mat.append(0)
                neighbors_weights.append(0)
        if first_run == True:
            projection_list.append(neighbors_mat)
        projected_weights.append(neighbors_weights)

update_ProjWeight(projection_list, projected_weights)



In [15]:
print(projection_list[0])

[0, 0, 0, tensor([[0.9900, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.9900, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.9900, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.9900]]), 0, tensor([[0.1102, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.1102, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.1102, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.1102]]), 0, 0, 0, 0, 0, tensor([[2.6985, 0.0000, 0.0000, 0.0000],
        [0.0000, 2.6985, 0.0000, 0.0000],
        [0.0000, 0.0000, 2.6985, 0.0000],
        [0.0000, 0.0000, 0.0000, 2.6985]]), 0, tensor([[1.2667, 0.0000, 0.0000, 0.0000],
        [0.0000, 1.2667, 0.0000, 0.0000],
        [0.0000, 0.0000, 1.2667, 0.0000],
        [0.0000, 0.0000, 0.0000, 1.2667]]), tensor([[1.8165, 0.0000, 0.0000, 0.0000],
        [0.0000, 1.8165, 0.0000, 0.0000],
        [0.0000, 0.0000, 1.8165, 0.0000],
        [0.0000, 0.0000, 0.0000, 1.8165]]), 0, 0, 0, 0, 0]


In [16]:
def testing(model, dataset, bs, criterion): 
    test_loss = 0
    correct = 0
    test_loader = DataLoader(MyDataset(dataset["features"], dataset["label"]), batch_size=bs)
    l = len(test_loader)
    model.eval()
    for data, labels in test_loader:
        data, labels = data, labels
        output = model(data)
        loss = criterion(output, labels)
        test_loss += loss.item()*data.size(0)
        #_, pred = torch.max(output, 1)
        #correct += pred.eq(labels.data.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    
    return test_loss

In [17]:
def rel_error(model):
    return (torch.norm(parameters_to_vector(model.parameters()) - datapoints[model.user_id]['exact_weights']) / torch.norm(datapoints[model.user_id]['exact_weights'])).detach()

In [18]:
torch.norm(datapoints[model.user_id]['exact_weights'])

tensor(5.0990, dtype=torch.float64)

In [19]:
model = MLP_Net(user_id=0)

from torch.nn.utils import parameters_to_vector, vector_to_parameters

with torch.no_grad():    
    params = parameters_to_vector(model.parameters())

    print(params)

params *= 2.

vector_to_parameters(parameters=model.parameters(), vec=params)

parameters_to_vector(model.parameters())





tensor([-0.0088, -0.1648,  0.3889, -0.4318])


tensor([-0.0175, -0.3297,  0.7778, -0.8636], grad_fn=<CatBackward0>)

In [20]:
#global_model = CNN_Net().cuda()
models = [MLP_Net(user_id=i) for i in range(no_users)]
dummy_models = [MLP_Net(user_id=i) for i in range(no_users)]

#model.load_state_dict(global_model.state_dict())

criterion = nn.MSELoss()


train_loss = []
test_loss = []
test_accuracy = []
total_rel_error = []

for curr_round in tqdm(range(1, it+1)):
    w, local_loss = [], []

    
    for i in range(no_users):
        dummy_models[i].load_state_dict(models[i].state_dict())
        local_update = ClientUpdate(dataset=datapoints[i], batchSize=batch_size, alpha=alpha, lamda=lamda, epochs=1, projection_list=projection_list, projected_weights=projected_weights)
        weights, loss = local_update.train(dummy_models[i])
        w.append(weights)
        local_loss.append(loss)
        models[i].load_state_dict(w[i])
        
    
    
    # Update prjection matrix
    
    #print(projection_list[0], projected_weights[0])
    
    for i in range(no_users):
        weights = parameters_to_vector(models[i].parameters())
        for j in G.neighbors(i):
            mat_vec_sum = torch.zeros_like(weights)
            for k in G.neighbors(i):
                mat_vec_sum = torch.add(mat_vec_sum, projected_weights[k][i] - projected_weights[i][k])
            #print(torch.outer(mat_vec_sum, weights))

            projection_list[i][j] = torch.add(projection_list[i][j], -1 * eta * lamda * mat_vec_sum)
                                         
    projected_weights = []                                          
    update_ProjWeight(projection_list, projected_weights, first_run=False)
        
        
        
    
    




          
            

    local_test_acc = []
    local_test_loss = []
    user_rel_error = 0
    for k in range(no_users):
      
        g_loss = testing(models[i], datapoints[i], 50, criterion)
        local_test_loss.append(g_loss)
        #user_rel_error += rel_error(models[i])
    
    
        

    g_loss = sum(local_test_loss) / len(local_test_loss)
    #total_rel_error.append(user_rel_error / no_users)
    
    

    test_loss.append(g_loss)
    #test_accuracy.append(g_accuracy)
    print("Training_loss %2.5f"% (test_loss[-1]))

  0%|          | 1/2000 [00:00<06:58,  4.77it/s]

Training_loss 28.90711


  0%|          | 2/2000 [00:00<07:44,  4.30it/s]

Training_loss 28.64864


  0%|          | 4/2000 [00:00<06:48,  4.89it/s]

...

Training_loss 0.00798
Training_loss 0.00796
Training_loss 0.00795


100%|█████████▉| 1999/2000 [04:11<00:00, 10.93it/s]

Training_loss 0.00792
Training_loss 0.00790


100%|██████████| 2000/2000 [04:11<00:00,  7.95it/s]

Training_loss 0.00786


In [21]:
#plot.plot(test_loss)
parameters_to_vector(models[19].parameters())

tensor([-1.9730,  1.9617,  3.0086, -2.9190], grad_fn=<CatBackward0>)

In [22]:
for j in G.neighbors(0):
    print(j)

3
5
11
13
14


In [23]:
parameters_to_vector(models[0].parameters())

tensor([1.9362, 1.9564, 2.9383, 2.9599], grad_fn=<CatBackward0>)

In [24]:
projection_list[0]

[0,
 0,
 0,
 tensor([[ 0.9609,  0.0036,  0.0077, -0.0459],
         [-0.0291,  0.9936,  0.0077, -0.0459],
         [-0.0291,  0.0036,  0.9977, -0.0459],
         [-0.0291,  0.0036,  0.0077,  0.9441]]),
 0,
 tensor([[ 0.0811,  0.0036,  0.0077, -0.0459],
         [-0.0291,  0.1138,  0.0077, -0.0459],
         [-0.0291,  0.0036,  0.1179, -0.0459],
         [-0.0291,  0.0036,  0.0077,  0.0643]]),
 0,
 0,
 0,
 0,
 0,
 tensor([[ 2.6695,  0.0036,  0.0077, -0.0459],
         [-0.0291,  2.7022,  0.0077, -0.0459],
         [-0.0291,  0.0036,  2.7063, -0.0459],
         [-0.0291,  0.0036,  0.0077,  2.6526]]),
 0,
 tensor([[ 1.2376,  0.0036,  0.0077, -0.0459],
         [-0.0291,  1.2703,  0.0077, -0.0459],
         [-0.0291,  0.0036,  1.2744, -0.0459],
         [-0.0291,  0.0036,  0.0077,  1.2208]]),
 tensor([[ 1.7874,  0.0036,  0.0077, -0.0459],
         [-0.0291,  1.8201,  0.0077, -0.0459],
         [-0.0291,  0.0036,  1.8241, -0.0459],
         [-0.0291,  0.0036,  0.0077,  1.7705]]),
 0,
 0,
 0

In [25]:
projected_weights[0]

[0,
 0,
 0,
 tensor([2.4637, 2.4207, 3.7774, 3.8030]),
 0,
 tensor([2.3529, 2.3177, 3.6755, 3.6197]),
 0,
 0,
 0,
 0,
 0,
 tensor([-2.3166,  1.5638,  2.5005, -3.2639]),
 0,
 tensor([ -6.8826,   6.3495,   9.7412, -10.2596]),
 tensor([-2.4359,  2.2153,  3.3876, -3.5679]),
 0,
 0,
 0,
 0,
 0]

In [26]:
test_loss = np.array(test_loss)
total_rel_error = np.array(total_rel_error)

In [27]:
print(test_loss)

[2.89071121e+01 2.86486397e+01 2.85130081e+01 ... 7.92254077e-03
 7.89543171e-03 7.85896205e-03]


In [28]:
np.save( 'training_loss_sheave_fml' + str(lamda).replace('.', '_'), test_loss)
#np.save('relative_error_sheave_fml' + str(lamda).replace('.', '_'), total_rel_error)